In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [2]:

git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b''
out : b'Already up to date.\n'


In [3]:
data_path='/home/resha/git/Lecture_Covid_19_data_analysis/data/raw/COVID-19/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38304,38324,38398,38494,38520,38544,38572,38606,38641,38716
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9967,10102,10255,10406,10553,10704,10860,11021,11185,11353
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,45773,46071,46364,46653,46938,47216,47488,47752,48007,48254
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1215,1215,1215,1261,1261,1301,1301,1344,1344,1344
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2876,2935,2965,2981,3033,3092,3217,3279,3335,3388


## Webscrapping

In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [8]:
all_rows=html_table.find_all('tr')

In [9]:
final_data_list=[]

In [10]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [11]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [12]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,45.323,+69,1.269,"11,5",1.868
3,Bayern,62.401,+427,2.324,"17,8",2.647
4,Berlin,12.354,+85,486,"13,0",226
5,Branden­burg,4.016,+10,63,"2,5",169
6,Bremen,2.137,+2,64,"9,4",59


## REST API calls

In [13]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [14]:
json_object=json.loads(data.content) 

In [15]:
type(json_object)

dict

In [16]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [17]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [18]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4255,1600120800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,146.890682,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6846,1600120800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,371.826965,2.089396e+09,4.188002e+05,268
2,3,03,Niedersachsen,Land,7982448,9,18023,1600120800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,225.782868,1.299836e+11,4.008988e+06,667
3,4,04,Bremen,Freie Hansestadt,682986,5,2137,1600120800000,04,4132268b-54de-4327-ac1e-760e915112f1,312.890747,1.119157e+09,3.357177e+05,59
4,5,05,Nordrhein-Westfalen,Land,17932651,10,62725,1600120800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,349.780967,8.782936e+10,2.648673e+06,1834


In [19]:

pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [20]:
pd_full_list.shape[0]

16

# API access via REST service, e.g. USA data 

example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [21]:


# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '2466440d7aa44d65b19082080426e4fb',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

ConnectionError: HTTPSConnectionPool(host='api.smartable.ai', port=443): Max retries exceeded with url: /coronavirus/stats/US (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6ee4bb1580>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:

US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [ ]:
print(json.dumps(US_dict,indent=2)) #string dump

# Individual States US

In [ ]:
US_dict['stats']['breakdowns'][0]

In [ ]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [ ]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)